# импорт данных

In [4]:
import pandas as pd

# 1. reinforcement_prices.csv - котировки метал сырья/продуктов
prices = pd.read_csv('/content/reinforcement_all_prices.csv', parse_dates=['value_date', 'dt'])
# print('PRICES:\n', prices.head(), '\n\n')

# 2. reinforecement_production.csv - производство метал сырья/продуктов
production = pd.read_csv('/content/reinforcement_production.csv', parse_dates=['value_date'])
# print('PRODUCTION:\n', production.head(), '\n\n')

# 3. domestic_supplies.csv - данные по поставкам сырья/продуктов
supplies = pd.read_csv('/content/reinforcement_domestic_supplies.csv', parse_dates=['value_date'])
# print('SUPPLIES:\n', supplies.head(), '\n\n')

# 4. reinforcement_cis_import.csv - импорт метал сырья/продуктов  СНГ
cis_import = pd.read_csv('/content/reinforcement_cis_import.csv', parse_dates=['value_date', 'dt'])
# print('CIS IMPORT:\n', cis_import.head(), '\n\n')

# 5. reinforcement_cis_export.csv - экспорт метал сырья/продуктов  СНГ
cis_export = pd.read_csv('/content/reinforcement_cis_export.csv', parse_dates=['value_date'])
# print('CIS EXPORT:\n', cis_export.head(), '\n\n')

# 6. reinforcement_consumption.csv- потребление метал сырья/продуктов
consumption = pd.read_csv('/content/reinforcement_consumption.csv', parse_dates=['value_date', 'dt'])
# print('CONSUMPTION:\n', consumption.head(), '\n\n')


# Проверим диапазоны дат
for name, df in zip(
    ['prices','production','supplies','cis_import','cis_export','consumption'],
    [prices, production, supplies, cis_import, cis_export, consumption]
):
    print(f"{name}: {df['value_date'].min()} — {df['value_date'].max()}")

# Если нужно, переименуем столбцы для единообразия (например, value_date → date)
for df in [prices, production, supplies, cis_import, cis_export, consumption]:
    df.rename(columns={'value_date':'date'}, inplace=True)


FileNotFoundError: [Errno 2] No such file or directory: '/content/reinforcement_production.csv'

In [ ]:
prices['product'].value_counts()

,count
product,
Арматура,16104
Лом,8784
ЖРС,6588
Чугун,4026
Концентрат коксующегося угля,4026
Квадратная заготовка,1830
ГБЖ,732
Кокс,366


In [ ]:
# Предположим, что вас интересует price только по определенному продукту:
# Например, арматура 12 мм, если это ваш целевой продукт
prices_12mm = prices[prices['product'] == 'Арматура'].copy()

# Убедитесь, что 'date' уже индекс
if 'date' in prices_12mm.columns:
    prices_12mm['date'] = pd.to_datetime(prices_12mm['date'])
    prices_12mm.set_index('date', inplace=True)

# Теперь ресемплируем только по числовому столбцу 'value'
weekly_prices = prices_12mm[['value']].resample('W-MON').mean().reset_index()

In [ ]:
# Список всех датафреймов, которые вы используете
dfs = [prices, production, supplies, cis_import, cis_export, consumption,weekly_prices]

# Установим date как индекс и убедимся, что он в формате datetime
for df in dfs:
    if 'date' in df.columns:
        df['date'] = pd.to_datetime(df['date'])
        df.set_index('date', inplace=True)
    else:
        print(f"У датафрейма {df} нет колонки 'date'!")

# Теперь date — это индекс во всех датафреймах.
# Пример проверки:
# print(prices.index)
# print(production.index)

In [ ]:
import pandas as pd
import numpy as np

# Примем, что датафреймы уже загружены и date - индекс

def monthly_to_weekly_with_lag(df, lag_months, value_col='value'):
    """
    Преобразовать месячные данные к недельной частоте с учетом задержки lag_months.
    Для каждой недели возвращает последнее доступное значение с учетом лага.
    """
    # Делаем date индексом, если еще нет
    if df.index.name != 'date':
        df = df.set_index('date')

    # Сохраняем только последний день месяца для каждого месяца
    monthly = df.resample('M').last()

    # Смещаем дату публикации на lag_months вперед
    monthly['publish_date'] = monthly.index + pd.DateOffset(days=lag_months*30)

    # Теперь для каждой недели определим, какое значение реально было известно на тот момент
    # Создадим датафрейм с неделями
    weekly_index = pd.date_range(monthly.index.min(), monthly.index.max() + pd.DateOffset(months=lag_months), freq='W-MON')
    out = pd.DataFrame(index=weekly_index)

    # Для каждой недели ищем последнее доступное значение
    out[value_col] = np.nan
    for i, week in enumerate(out.index):
        # Все записи, опубликованные до этой недели
        avail = monthly[monthly['publish_date'] <= week]
        if len(avail) > 0:
            out.iloc[i, 0] = avail[value_col].iloc[-1]
    return out

# Пример: для производства (лаг 2 месяца)
weekly_prod_lag = monthly_to_weekly_with_lag(production[['value']], lag_months=2, value_col='value')
weekly_prod_lag = weekly_prod_lag.rename(columns={'value': 'production_lag2m'})

# Для импорта/экспорта лаг может отличаться (например, 1 месяц):
weekly_imp_lag = monthly_to_weekly_with_lag(cis_import[['value']], lag_months=1, value_col='value').rename(columns={'value':'import_lag1m'})
weekly_exp_lag = monthly_to_weekly_with_lag(cis_export[['value']], lag_months=1, value_col='value').rename(columns={'value':'export_lag1m'})


<ipython-input-8-4747427690f2>:16: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly = df.resample('M').last()
<ipython-input-8-4747427690f2>:16: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly = df.resample('M').last()
<ipython-input-8-4747427690f2>:16: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly = df.resample('M').last()


In [ ]:
# Если после .reset_index() у вас date снова стал колонкой:
if 'date' in target.columns:
    target['date'] = pd.to_datetime(target['date'])
    target.set_index('date', inplace=True)

# Теперь можно:
# target['month'] = target.index.month
# target['weekofyear'] = target.index.isocalendar().week.astype(int)


NameError: name 'target' is not defined

In [ ]:
# Предположим, что weekly_prices уже есть, дата - индекс
target = weekly_prices.copy()
target = target.sort_index()

# Лаги целевой переменной
for lag in [1, 2, 4]:
    target[f'price_lag_{lag}w'] = target['value'].shift(lag)

# Скользящие средние и темпы прироста
target['price_ma_4w'] = target['value'].rolling(4).mean()
target['price_return_1w'] = target['value'].pct_change(1)
target['price_volatility_4w'] = target['value'].rolling(4).std()

if 'date' in target.columns:
    target['date'] = pd.to_datetime(target['date'])
    target.set_index('date', inplace=True)
# Сезонные признаки
target['month'] = target.index.month
target['weekofyear'] = target.index.isocalendar().week.astype(int)

# Праздники (пример — добавим 1 января и 9 мая как dummy)
holidays = pd.to_datetime(['2018-01-01', '2018-05-09', '2019-01-01', '2019-05-09', '2020-01-01', '2020-05-09',
                           '2021-01-01', '2021-05-09', '2022-01-01', '2022-05-09', '2023-01-01', '2023-05-09'])
target['is_holiday'] = target.index.isin(holidays).astype(int)
display(target)
# Внешние шоки (ковид, санкции)
covid_start = pd.to_datetime('2020-03-15')
sanction_start = pd.to_datetime('2022-02-24')
target['covid'] = (target.index >= covid_start).astype(int)
target['sanctions'] = (target.index >= sanction_start).astype(int)

# Агрегаты по производству/импорту/экспорту (рост/падение)
target = target.join(weekly_prod_lag)
target = target.join(weekly_imp_lag)
target = target.join(weekly_exp_lag)

# Темпы прироста по производству/импорту
target['prod_return'] = target['production_lag2m'].pct_change()
target['import_return'] = target['import_lag1m'].pct_change()
target['export_return'] = target['export_lag1m'].pct_change()

# Можно добавить rolling mean/std для этих признаков

# Итоговый датафрейм
features = target.dropna(how='all').copy()  # удаляем строки без данных (например, из-за лагов)



,value,price_lag_1w,price_lag_2w,price_lag_4w,price_ma_4w,price_return_1w,price_volatility_4w,month,weekofyear,is_holiday
date,,,,,,,,,,
2018-01-01,32131.363636,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1
2018-01-08,32350.909091,32131.363636,NaN,NaN,NaN,0.006833,NaN,1,2,0
2018-01-15,32286.818182,32350.909091,32131.363636,NaN,NaN,-0.001981,NaN,1,3,0
2018-01-22,31905.000000,32286.818182,32350.909091,NaN,32168.522727,-0.011826,198.396594,1,4,0
2018-01-29,31968.636364,31905.000000,32286.818182,32131.363636,32127.840909,0.001995,223.634631,1,5,0
...,...,...,...,...,...,...,...,...,...,...
2024-12-02,59464.090909,59627.954545,59802.954545,59839.318182,59679.034091,-0.002748,167.689223,12,49,0
2024-12-09,59382.272727,59464.090909,59627.954545,59821.136364,59569.318182,-0.001376,186.264109,12,50,0
2024-12-16,59273.181818,59382.272727,59464.090909,59802.954545,59436.875000,-0.001837,149.475711,12,51,0


In [ ]:
from sklearn.model_selection import TimeSeriesSplit

# Разделим на train/test: например, последние 52 недели — test (год)
test_size = 52
features = features.sort_index()
train = features.iloc[:-test_size]
test = features.iloc[-test_size:]

print(f"Train: {train.index.min()} — {train.index.max()}")
print(f"Test:  {test.index.min()} — {test.index.max()}")

# Для кросс-валидации по времени
tscv = TimeSeriesSplit(n_splits=5)
for fold, (train_idx, val_idx) in enumerate(tscv.split(train)):
    print(f"Fold {fold}: Train [{train.index[train_idx[0]]} — {train.index[train_idx[-1]]}], "
          f"Val [{train.index[val_idx[0]]} — {train.index[val_idx[-1]]}]")

# Пример выделения X, y
feature_cols = [c for c in features.columns if c not in ['value']]
X_train, y_train = train[feature_cols], train['value']
X_test, y_test = test[feature_cols], test['value']


Train: 2018-01-01 00:00:00 — 2024-01-01 00:00:00
Test:  2024-01-08 00:00:00 — 2024-12-30 00:00:00
Fold 0: Train [2018-01-01 00:00:00 — 2019-01-07 00:00:00], Val [2019-01-14 00:00:00 — 2020-01-06 00:00:00]
Fold 1: Train [2018-01-01 00:00:00 — 2020-01-06 00:00:00], Val [2020-01-13 00:00:00 — 2021-01-04 00:00:00]
Fold 2: Train [2018-01-01 00:00:00 — 2021-01-04 00:00:00], Val [2021-01-11 00:00:00 — 2022-01-03 00:00:00]
Fold 3: Train [2018-01-01 00:00:00 — 2022-01-03 00:00:00], Val [2022-01-10 00:00:00 — 2023-01-02 00:00:00]
Fold 4: Train [2018-01-01 00:00:00 — 2023-01-02 00:00:00], Val [2023-01-09 00:00:00 — 2024-01-01 00:00:00]


In [ ]:
X_train

,price_lag_1w,price_lag_2w,price_lag_4w,price_ma_4w,price_return_1w,price_volatility_4w,month,weekofyear,is_holiday,covid,sanctions,production_lag2m,import_lag1m,export_lag1m,prod_return,import_return,export_return
date,,,,,,,,,,,,,,,,,
2018-01-01,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-08,32131.363636,NaN,NaN,NaN,0.006833,NaN,1,2,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-15,32350.909091,32131.363636,NaN,NaN,-0.001981,NaN,1,3,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-22,32286.818182,32350.909091,NaN,32168.522727,-0.011826,198.396594,1,4,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-29,31905.000000,32286.818182,32131.363636,32127.840909,0.001995,223.634631,1,5,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-04,64921.136364,64955.113636,67341.136364,64327.897727,-0.053593,1987.384685,12,49,0,1,1,0.0,0.0,0.0,NaN,NaN,NaN
2023-12-11,61441.818182,64921.136364,65993.522727,63099.119318,-0.005915,2128.722457,12,50,0,1,1,0.0,0.0,0.0,NaN,NaN,NaN
2023-12-18,61078.409091,61441.818182,64955.113636,61966.875000,-0.010679,2013.834823,12,51,0,1,1,0.0,0.0,0.0,NaN,NaN,NaN


In [ ]:
X_train = X_train.set_index('date')

KeyError: "None of ['date'] are in the columns"

# базовые модели

In [ ]:
import numpy as np

# Наивная модель: прогноз — последнее значение train
naive_pred = np.repeat(y_train.iloc[-1], len(y_test))

In [ ]:
window = 7  # например, неделя
moving_avg_pred = y_train.rolling(window).mean().iloc[-1]
moving_avg_pred = np.repeat(moving_avg_pred, len(y_test))

In [ ]:
import numpy as np

# Заменяем inf на 0
X_train = X_train.replace([np.inf, -np.inf], 0)

# Заполняем NaN средним (или 0, или медианой)
X_train = X_train.fillna(X_train.mean())

# Если хотите — можно удалить строки с NaN:
# mask = ~X_train.isnull().any(axis=1)
# X_train = X_train[mask]
# y_train = y_train[mask]

model = sm.tsa.SARIMAX(y_train, exog=X_train, order=(1,1,1), seasonal_order=(1,1,1,52))
sarimax = model.fit(disp=False)


NameError: name 'sm' is not defined

In [ ]:
X_train.isnull().sum(), np.isinf(X_train.values).sum()

(price_lag_1w             0
 price_lag_2w             0
 price_lag_4w             0
 price_ma_4w              0
 price_return_1w          0
 price_volatility_4w      0
 is_holiday               0
 covid                    0
 sanctions                0
 production_lag2m       314
 import_lag1m           314
 export_lag1m           314
 prod_return            314
 import_return          314
 export_return          314
 dtype: int64,
 np.int64(0))

In [ ]:
X_train.shape

(314, 15)

In [ ]:
import statsmodels.api as sm

# SARIMAX (можно и без экзогенных признаков)
model = sm.tsa.SARIMAX(y_train, exog=X_train, order=(1,1,1), seasonal_order=(1,1,1,52))
sarimax = model.fit(disp=False)
sarimax_pred = sarimax.predict(start=y_test.index[0], end=y_test.index[-1], exog=X_test)


MissingDataError: exog contains inf or nans

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor

# Linear Regression
lr = LinearRegression().fit(X_train, y_train)
lr_pred = lr.predict(X_test)

# Ridge
ridge = Ridge().fit(X_train, y_train)
ridge_pred = ridge.predict(X_test)

# Lasso
lasso = Lasso().fit(X_train, y_train)
lasso_pred = lasso.predict(X_test)

# ElasticNet
enet = ElasticNet().fit(X_train, y_train)
enet_pred = enet.predict(X_test)

# Random Forest
rf = RandomForestRegressor(n_estimators=100, random_state=42).fit(X_train, y_train)
rf_pred = rf.predict(X_test)

# XGBoost
xgb_reg = xgb.XGBRegressor(n_estimators=100, random_state=42).fit(X_train, y_train)
xgb_pred = xgb_reg.predict(X_test)

# LightGBM
lgb_reg = lgb.LGBMRegressor(n_estimators=100, random_state=42).fit(X_train, y_train)
lgb_pred = lgb_reg.predict(X_test)

# CatBoost
cat = CatBoostRegressor(verbose=0, random_state=42).fit(X_train, y_train)
cat_pred = cat.predict(X_test)


Prophet

In [ ]:
from prophet import Prophet
prophet_df = y_train.reset_index().rename(columns={'index': 'ds', 'value': 'y'})
m = Prophet()
m.fit(prophet_df)
future = m.make_future_dataframe(periods=len(y_test), freq='W')
forecast = m.predict(future)
prophet_pred = forecast['yhat'].iloc[-len(y_test):].values


In [ ]:
from sklearn.ensemble import StackingRegressor

estimators = [
    ('rf', RandomForestRegressor(n_estimators=50, random_state=42)),
    ('lasso', Lasso()),
    ('cat', CatBoostRegressor(verbose=0, random_state=42))
]
stack = StackingRegressor(estimators=estimators, final_estimator=LinearRegression())
stack.fit(X_train, y_train)
stack_pred = stack.predict(X_test)


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

def smape(y_true, y_pred):
    return 100*np.mean(2*np.abs(y_pred - y_true) / (np.abs(y_pred) + np.abs(y_true)))

models = {
    'Naive': naive_pred,
    'MovingAvg': moving_avg_pred,
    'SARIMAX': sarimax_pred,
    'LR': lr_pred,
    'Ridge': ridge_pred,
    'Lasso': lasso_pred,
    'ElasticNet': enet_pred,
    'RF': rf_pred,
    'XGB': xgb_pred,
    'LGB': lgb_pred,
    'CatBoost': cat_pred,
    'Prophet': prophet_pred,
    'Stack': stack_pred
}

for name, pred in models.items():
    print(f'{name:10s}: MAE={mean_absolute_error(y_test, pred):.2f} '
          f'RMSE={mean_squared_error(y_test, pred, squared=False):.2f} '
          f'sMAPE={smape(y_test, pred):.2f}')


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(20,6))
plt.plot(y_test.index, y_test, label='Fact')
for name, pred in models.items():
    plt.plot(y_test.index, pred, label=name)
plt.title('Forecast vs Fact')
plt.legend()
plt.show()


In [ ]:
def forecast_next_week(model, last_data):
    # last_data — DataFrame с последними известными признаками
    return model.predict(last_data)

next_week_pred = forecast_next_week(cat, X_test.iloc[[-1]])  # пример для CatBoost
print('Next week forecast:', next_week_pred)
